# Modelling

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.core.display import display, HTML
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore')

In [3]:
X =pd.read_csv('../data/interim/explantorydf.csv', index_col=0) #read in features and target(s)
y = pd.read_csv('../data/interim/targetdf.csv', index_col=0)

In [4]:
X.head()

,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Speed_limit,Age_of_Vehicle,Engine_Capacity_.CC.,Vehicle_Reference,1st_Road_Class_A,1st_Road_Class_A(M),1st_Road_Class_B,1st_Road_Class_C,...,Month_of_Accident_3,Month_of_Accident_4,Month_of_Accident_5,Month_of_Accident_6,Month_of_Accident_7,Month_of_Accident_8,Month_of_Accident_9,Month_of_Accident_10,Month_of_Accident_11,Month_of_Accident_12
1,-0.070477,2.354227,-0.66806,-0.641979,3.495725,-0.712284,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.070477,-0.400577,-0.66806,-0.232590,3.513504,-0.712284,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,-0.070477,-0.400577,-0.66806,-0.437285,-0.115081,-0.712284,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y.head(3)


,Fatal,Serious,Slight
1,0,0,1
2,0,0,1
3,0,0,1


In [6]:
print(X.shape, y.shape)

(2055877, 214) (2055877, 3)


In [31]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size =.30)


# Preparing Model

Multiple Models to Try:

Decision Tree

Random Forest

XGBoost

Cross Validation

Grid Search

Pipeline

>>> parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
>>> svc = svm.SVC()
>>> clf = GridSearchCV(svc, parameters)
>>> clf.fit(iris.data, iris.target)


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import time

In [27]:
param_grid = {'n_estimators':np.arange(1,30),
              'max_depth':np.arange(1,7)}

In [28]:
clf = RandomForestClassifier()
clf_cv = GridSearchCV(clf, param_grid, cv=5)

In [34]:
start = time.process_time()
clf_cv.fit(X_train, y_train)
print(time.process_time() - start, 'seconds')

13658.625 seconds


In [35]:
clf_cv.best_params_

{'max_depth': 1, 'n_estimators': 1}

In [36]:
clf_cv.best_score_

0.8575094518719449

In [37]:
np.arange(1,30)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [38]:
y_pred = clf_cv.predict(X_test)

In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [49]:
#y_test_unencoded = np.argmax(y_test, axis=1)
y_pred_unencoded = np.argmax(y_pred, axis=1)

In [60]:
y_test_array = np.array(y_test)
y_test_unencoded = np.argmax(y_test_array, axis=1)

In [62]:
confusion_matrix(y_test_unencoded, y_pred_unencoded)

array([[     0,      0,   8270],
       [     0,      0,  79130],
       [     0,      0, 529364]], dtype=int64)

In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8270
           1       0.00      0.00      0.00     79130
           2       0.86      1.00      0.92    529364

   micro avg       0.86      0.86      0.86    616764
   macro avg       0.29      0.33      0.31    616764
weighted avg       0.74      0.86      0.79    616764
 samples avg       0.86      0.86      0.86    616764



In [78]:
y_train_fatal = y_train['Fatal']
y_test_fatal = y_test['Fatal']

In [88]:
clf = RandomForestClassifier(n_jobs=-1)

In [89]:
clf.fit(X_train, y_train_fatal)

RandomForestClassifier(n_jobs=-1)

In [90]:
y_pred_fatal  =clf.predict(X_test)

In [91]:
print(confusion_matrix(y_test_fatal, y_pred_fatal))

[[608488      6]
 [  8228     42]]


In [92]:
print(classification_report(y_test_fatal, y_pred_fatal))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    608494
           1       0.88      0.01      0.01      8270

    accuracy                           0.99    616764
   macro avg       0.93      0.50      0.50    616764
weighted avg       0.99      0.99      0.98    616764

